# Classificação de imagens de raio-x (CXR8)

## Importação dos pacotes
***
- `Importação dos pacotes para preparação de dados`;

In [34]:
# == preparação dos dados == 
from cxr8_dados import data_download, organize_csv, download_images, train_validation_test_split 
from keras.preprocessing.image import ImageDataGenerator

# == definição do modelo ==
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

# == verificação do modelo
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

# ignorando avisos de pacotes desatualizados  
import warnings
warnings.filterwarnings("ignore")

## Preparação dos dados
***
- `Fazer o download do dataframe e das imagens de radiografia`;  
- `Organizar o dataframe e definir um diretório com as imagens`;
- `Particionar o dataframe em dados de treinamento, validação e teste`;
- `Analisar a distrubuição dos dados em cada uma das partições`;
- `Preparar um gerador de dados de treinamento, validação e teste a partir do dataframe`;  

In [35]:
# baixando dataframe
data_download('https://raw.githubusercontent.com/Alyssonmach/cnn-lung-diseases/main/dataframe-info.csv', 'dataframe-info.csv')

In [36]:
# baixando as imagens a serem utilizadas
download_images()

In [37]:
# descompactando os arquivos e removendo os arquivos compactados
!tar -xvzf images_01.tar.gz -C /content/
!rm /content/images_01.tar.gz 
!tar -xvzf images_02.tar.gz -C /content/
!rm /content/images_02.tar.gz
!tar -xvzf images_03.tar.gz -C /content/
!rm /content/images_03.tar.gz
!tar -xvzf images_04.tar.gz -C /content/
!rm /content/images_04.tar.gz
!tar -xvzf images_05.tar.gz -C /content/
!rm /content/images_05.tar.gz
!tar -xvzf images_06.tar.gz -C /content/
!rm /content/images_06.tar.gz
!tar -xvzf images_07.tar.gz -C /content/
!rm /content/images_07.tar.gz
!tar -xvzf images_08.tar.gz -C /content/
!rm /content/images_08.tar.gz
!tar -xvzf images_09.tar.gz -C /content/
!rm /content/images_09.tar.gz
!tar -xvzf images_10.tar.gz -C /content/
!rm /content/images_10.tar.gz
!tar -xvzf images_11.tar.gz -C /content/
!rm /content/images_11.tar.gz
!tar -xvzf images_12.tar.gz -C /content/
!rm /content/images_12.tar.gz

In [38]:
# especificando o diretório com as imagens 
IMAGE_DIR = "/content/images/"

In [39]:
# obtendo o dataframe organizando dataframe = organize_csv('/content/dataframe-info.csv')

In [40]:
# visualizando o dataframe
print(dataframe.head())
print('dataframe shape:', dataframe.shape)

        Image Index  labels
0  00003683_006.png       0
1  00017620_002.png       0
2  00016074_011.png       1
3  00005218_003.png       1
4  00022677_005.png       0
dataframe shape: (48747, 2)


In [41]:
# particionando o dataset em dados de treino, validação e teste  
train_df, validation_df, test_df = train_validation_test_split(dataframe)

In [42]:
# visualizando detalhes dos dados de treinamento
print(train_df.head())
print('train_df shape:', train_df.shape)

            Image Index  labels
2947   00019140_001.png       1
29173  00005261_002.png       1
42712  00001265_000.png       0
25474  00013930_002.png       1
4944   00022093_000.png       1
train_df shape: (39484, 2)


In [43]:
# visualizando detalhes dos dados de treinamento
print(validation_df.head())
print('validation_df shape:', validation_df.shape)

            Image Index  labels
8289   00007474_000.png       0
2044   00001829_000.png       0
40923  00006605_010.png       1
20813  00029801_003.png       1
25273  00005140_002.png       0
validation_df shape: (4388, 2)


In [44]:
# visualizando detalhes dos dados de teste 
print(test_df.head())
print('validation_df shape:', test_df.shape)

            Image Index  labels
31843  00015071_000.png       1
36040  00023933_000.png       0
45886  00017225_005.png       1
5326   00011460_037.png       1
40405  00021586_000.png       1
validation_df shape: (4875, 2)


In [45]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(samplewise_center = True, samplewise_std_normalization = True)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      directory = IMAGE_DIR,
                                                      x_col = 'Image Index',
                                                      y_col = 'labels',
                                                      batch_size = 256,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'raw',
                                                      color_mode = 'rgb',
                                                      target_size = (256, 256))
# criando o gerador de imagens de validação 
valid_generator = image_generator.flow_from_dataframe(
                                                      dataframe = validation_df,
                                                      directory = IMAGE_DIR, 
                                                      x_col = 'Image Index',
                                                      y_col = 'labels',
                                                      batch_size = 128,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'raw',
                                                      target_size = (256, 256))

# normalizando as imagens de teste 
test_datagen = ImageDataGenerator(samplewise_center = True, samplewise_std_normalization = True)

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  directory = IMAGE_DIR,
                                                  x_col = 'Image Index',
                                                  y_col = 'labels',
                                                  batch_size = 128,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'raw',
                                                  target_size = (256, 256))

Found 39484 validated image filenames.
Found 4388 validated image filenames.
Found 4875 validated image filenames.


## Definição do modelo

In [46]:
# definindo o local do arquivo em que os pesos devem ser salvos 
filepath = "transferlearning_weights.hdf5" 
# definindo um callback de checkpoint para salvar os pesos durante o treinamento  
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [47]:
# definindo um callback de redução da taxa de aprendizado caso a rede entre em um platô  
lr_reduce = ReduceLROnPlateau(monitor = 'val_acc', factor = 0.1, min_delta = 1e-5, patience = 5, verbose = 1)

In [48]:
# definindo uma lista de callbacks
callbacks = [checkpoint, lr_reduce] 

In [49]:
# transferência de aprendizado
conv_base = VGG19(weights = 'imagenet', include_top = False, input_shape = (256, 256, 3))

In [50]:
# visualizando o a arquitetura VGG19 utilizada 
conv_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [51]:
# definindo inicialmente toda rede como treinável 
conv_base.trainable = True
# criando uma flag 
set_trainable = False

# iterando entre as camadas da rede convolucional VGG19 
for layer in conv_base.layers:
    # caso estiver na última camada... 
    if layer.name == 'block5_conv1':
        # configure-a como treinável 
        set_trainable = True
    
    # caso a flag for verdadeira (na última camada)...
    if set_trainable:
        # mantenha a camada treinável 
        layer.trainable = True
    # caso a flag seja falsa (demais camadas)... 
    else:
        # mantenha as características aprendidas pelas camadas (não treináveis) 
        layer.trainable = False

In [52]:
# observando a arquitetura da rede após o refinamento
conv_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [53]:
# definindo um modelo sequencial de rede neural 
model = models.Sequential()
# adicionando toda a arquitetura da VGG19 após o refinamento 
model.add(conv_base)
# adicionando uma camada de pooling (método do maior píxel) 
model.add(layers.MaxPooling2D())
# aplicando uma camada de normalização para otimizar a rede 
model.add(layers.BatchNormalization())
# aplicando uma camada de achatamento para conectar as camadas de convolução a uma rede neural densa 
model.add(layers.Flatten())
# definindo uma camada densa com 128 neurônios e uma função de ativação relu
model.add(layers.Dense(units = 128, activation = tf.nn.relu))
# aplicando uma camada de normalização na rede neural densa (zera 20% dos neurônios da camada anterior)
model.add(layers.Dropout(rate = 0.2))
# aplicando uma camada de saída com 15 unidades e uma função de ativação softmax 
model.add(layers.Dense(units = 2, activation = tf.nn.softmax)) 

In [54]:
# visualizando a nova arquitetura definida baseada em transferência de aprendizado
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 8, 8, 512)         20024384  
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 512)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 4, 512)         2048      
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1048704   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [55]:
# compilando o modelo
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(lr = 5e-3), metrics = ['acc'])

## Treinamento do modelo

In [ ]:
# treinando a rede 
history = model.fit_generator(train_generator,
                              steps_per_epoch = 39484 // 256, 
                              validation_data = valid_generator,
                              validation_steps = 4388 // 128,
                              callbacks = callbacks, epochs = 50, verbose = 1)

Epoch 1/50
 26/154 [====>.........................] - ETA: 9:43:49 - loss: 2.7432 - acc: 0.5370

## Verificação do modelo